In [1]:
'''
IMPORTS
'''

%matplotlib inline

# Standard imports
import copy
import itertools

# Scientific computing imports
import numpy
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from operator import itemgetter
from heapq import heappush, heappop
from itertools import count

In [2]:
class MultiPlex(object):
    def __init__(self):
        self.plex=[]
        self.name=[]
        self.global_node={}
        
    def add_plex(self,name):
        self.plex.append(nx.Graph())
        self.name.append(name)
        #print self.plex
        if len(self.plex)>1:
            for node in self.plex[-2].nodes():
                print node,self.plex[-2].nodes()
                if self.plex[-2].node[node]:
                    self.plex[-1].add_node(node,self.plex[-2].node(node))
                else:
                    self.plex[-1].add_node(node)
                
        
    def add_plex_node(self,node_name,data={}):        
        self.global_node[node_name]=data
        for plex in self.plex:
            if node_name in plex.nodes(): #node exists
                print 'node exists'
                break
            else:
                plex.add_node(node_name,data)
                
    def remove_plex_node(self,node_name):
        self.global_node.pop(node_name,None)
        for plex in self.plex:
            plex.remove_node(node_name)
                    
    def edit_plex_node(self,node_name,node_data):
        if node_name not in self.global_node:
            print 'node does not exist'
            return
        else:
            for plex in self.plex:
                plex.node[node_name]=node_data
                
            self.global_node[node_name]=node_data
    
    def add_plex_edge(self,(u,v,plex),data={}):
        #check if u and v exist
        if plex not in self.name:
            print 'plex does not exist'
            return
        elif (u not in self.global_node) or (v not in self.global_node):
            print u, 'or', v, 'does not exist'
            return
        else:
            plex_index=self.name.index(plex)
            self.plex[plex_index].add_edge(u,v,data)
        
                
    def plexes(self):
        return iter(self.plex)
    
    def names(self):
        return iter(self.name)
    
    def global_nodes(self):
        return iter(self.global_node)
                
 
    def __repr__(self):
        skip_none = True
        repr_string = type(self).__name__ + " ["
        except_list = "model"

        elements = [e for e in dir(self) if str(e) not in except_list]
        for e in elements:
            # Make sure we only display "public" fields; skip anything private (_*), that is a method/function, or that is a module.
            if not e.startswith("_") and eval('type(self.{0}).__name__'.format(e)) not in ['DataFrame', 'function', 'method', 'builtin_function_or_method', 'module', 'instancemethod']:
                    value = eval("self." + e)
                    if value != None and skip_none == True:
                        repr_string += "{0}={1}, ".format(e, value)

        # Clean up trailing space and comma.
        return repr_string.strip(" ").strip(",") + "]"
        

In [4]:
class ShipNET(object):
    def __init__(self,initial_grid_size):
        self.initial_grid_size=initial_grid_size
        self.ship=nx.grid_graph(dim=initial_grid_size)
        self.layout=nx.spectral_layout(self.ship)
        
        #label streets and penetrations
        self.dual_streets={}
        self.geometric_characteristics={'H':'horizontal','L':'longitudinal','T':'transverse'}
        structural_denotation=['T','L','H']
        self.transfer_types={'P':None,'EM_P':None,'C':None,'I':None,'H':None}
        #self.transfer_types={'E':None,'M':None,'I':None,'H':None}
        for u,v in self.ship.edges():
            self.ship[u][v]['availability']=list(self.transfer_types)
        
        for i in range(len(self.initial_grid_size)):
            for structure in range(self.initial_grid_size[i]+1):
                s_name='{}{}'.format(structural_denotation[i],structure)
                self.dual_streets[s_name]={}
                self.dual_streets[s_name]['orientation']=self.geometric_characteristics[structural_denotation[i]]
                self.dual_streets[s_name]['availability']=list(self.transfer_types)
                
        for u,v in self.ship.edges():
            coord_change=[(a is b) for a, b in zip(u,v)].index(False)
            if coord_change==0: #move in x
                self.ship[u][v]['penetration']='T{}'.format(max(u[0],v[0]))
                self.ship[u][v]['streets']=['L{}'.format(u[1]),'L{}'.format((u[1]+1)),'H{}'.format(u[2]),'H{}'.format((u[2]+1))]   

            if coord_change==1: #move in y
                self.ship[u][v]['penetration']='L{}'.format(max(u[1],v[1]))
                self.ship[u][v]['streets']=['H{}'.format(u[2]),'H{}'.format((u[2]+1)),'T{}'.format(u[0]),'T{}'.format((u[0]+1))]

            if coord_change==2: #move in z
                self.ship[u][v]['penetration']='H{}'.format(max(u[2],v[2]))
                self.ship[u][v]['streets']=['T{}'.format(u[0]),'T{}'.format((u[0]+1)),'L{}'.format(u[1]),'L{}'.format((u[1]+1))]

            #print (u,v),self.ship[u][v]['streets'],self.ship[u][v]['penetration']

        #generate default information dual
        self.ship_dual=nx.Graph()
        for key in self.dual_streets:
            if self.dual_streets[key]['orientation'] is 'horizontal':
                for cross_street in self.dual_streets:
                    if self.dual_streets[cross_street]['orientation'] is not 'horizontal':
                        self.ship_dual.add_edge(key,cross_street)

            if self.dual_streets[key]['orientation'] is 'transverse':
                for cross_street in self.dual_streets:
                    if self.dual_streets[cross_street]['orientation'] is not 'transverse':
                        self.ship_dual.add_edge(key,cross_street)

            if self.dual_streets[key]['orientation'] is 'longitudinal':
                for cross_street in self.dual_streets:
                    if self.dual_streets[cross_street]['orientation'] is not 'longitudinal':
                        self.ship_dual.add_edge(key,cross_street)
                        
    def remove_ship_node(self,node_name):
        self.ship.remove_node(node_name)
    
    def remove_ship_edge(self,edge_name):
        self.ship.remove_edge(edge_name)
    
    def relative_position(self,structure,node):
        if self.dual_streets[structure]['orientation'] is 'transverse':
            key_coord=0
        if self.dual_streets[structure]['orientation'] is 'longitudinal':
            key_coord=1
        if self.dual_streets[structure]['orientation'] is 'horizontal':
            key_coord=2
        
        #print key_coord
             
        coord_value=[]

        for u,v in self.ship.edges():
            if structure in self.ship[u][v]['streets']:
                coord_value=u[key_coord]
                #print coord_value
                break
        return node[key_coord]>coord_value

    def transfer_graphs(self):
        for transfer in self.transfer_types:
            s_g=nx.Graph()
            for j,k,d in self.ship.edges(data=True):
                available_streets=[]
                #print (j,k),d
                if transfer in self.ship[j][k]['availability']:
                    for street in d['streets']:
                        if transfer in self.dual_streets[street]['availability']:
                            available_streets.append(street)
                    if available_streets:
                        s_g.add_edge(j,k,streets=available_streets)
            self.transfer_types[transfer]=s_g
            
    def set_dc_config(self,water_tight_bulkheads,dc_deck_level,exception=[]):
        self.num_bh=len(water_tight_bulkheads)
        self.height_dc=dc_deck_level
        dc_deck='H{}'.format(self.height_dc)
        for u,v in self.ship.edges():
            #print (u,v), self.relative_position(dc_deck,u)
            #print (u,v),(self.ship[u][v]['penetration'] in water_tight_bulkheads) and (not self.relative_position(dc_deck,u))
            if (self.ship[u][v]['penetration'] in water_tight_bulkheads) and (not self.relative_position(dc_deck,u)):
                self.ship[u][v]['availability']=exception
    
    def gen_disjoint_sets(self,affordance_multiplex,method,k=10):
        self.affordance_multiplex=affordance_multiplex
        self.transfer_graphs()
        self.disjoint_sets={}
        self.num_paths=k
        for plex,name in zip(self.affordance_multiplex.plexes(),self.affordance_multiplex.names()):
            for u,v in plex.edges():
                #print u,v,name
                #print u,self.affordance_multiplex.global_node[u]['loc']
                base_graph=self.transfer_types[name]
                n1=self.affordance_multiplex.global_node[u]['loc_possible'][-1]
                n2=self.affordance_multiplex.global_node[v]['loc_possible'][-1]
                if n1 not in self.disjoint_sets:
                    self.disjoint_sets[n1]={}
                if n2 not in self.disjoint_sets:
                    self.disjoint_sets[n2]={}
                if method=='shortest':
                    paths=nx.all_shortest_paths(base_graph, n1, n2)
                if method=='k-shortest':
                    k_paths=self.k_shortest_paths(base_graph, n1, n2, k=self.num_paths, weight='weight')
                    paths = [list(x) for x in set(tuple(x) for x in k_paths[1])]
                step_paths=[]
                for p in paths:
                    step_paths.append(p[1:-1])
                disjoint_paths=[]
                disjoint_paths_rev=[]
                cutoff=float("inf")
                if len(step_paths)>cutoff:
                    step_paths=itemgetter(*numpy.random.choice(range(len(step_paths)), cutoff, replace=False))(step_paths)
                for d_set in self.comb(step_paths):
                    d_set_rev=[]
                    for path_index in xrange(len(d_set)):
                        if not d_set[path_index]:
                            d_set[path_index].insert(0,n1)
                            d_set[path_index].append(n2)
                            if n1==n1:
                                d_set[path_index]=d_set[path_index][1:-1]
                        elif d_set[path_index][0]!=n1: #check if head and tail are appended
                            d_set[path_index].insert(0,n1)
                            d_set[path_index].append(n2)
                            if n1==n2:
                                d_set[path_index]=d_set[path_index][1:-1]
                    disjoint_paths.append(d_set)
                    for p in d_set:
                        d_set_rev.append(p[::-1])
                    disjoint_paths_rev.append(d_set_rev)
                
                self.disjoint_sets[n1][n2]=disjoint_paths
                self.disjoint_sets[n2][n1]=disjoint_paths_rev
    
    def affordance_routing(self,affordance_multiplex,num_arrangements,method='random',wander=1.5,redundancy=1):
        self.affordance_multiplex=affordance_multiplex
        self.num_arrangements=num_arrangements
        self.wander=wander
        self.redundancy=redundancy
        self.method=method
        #first get the available subgraph
        self.transfer_graphs()
            
        #create dataframe for results
        self.affordance_edges=[]
        for plex,name in zip(self.affordance_multiplex.plexes(),self.affordance_multiplex.names()):
            #print name
            self.affordance_nodes=plex.nodes()
            for u,v in plex.edges():
                #print edge,name
                self.affordance_edges.append((u,v,name))
        
        #print 'edges',self.affordance_edges
                
        self.arr_e_list_method=[]
        self.arr_n_list_method=[]

        fixed_loc=pd.DataFrame(index=self.ship.nodes(),columns=self.affordance_nodes)
        fixed_loc=fixed_loc.fillna(0.0)
        
        for n in self.affordance_multiplex.global_nodes():
            n_loc=self.affordance_multiplex.global_node[n]['loc_possible'][-1]
            fixed_loc.ix[n_loc,n]+=1
        
        #Set up disjoint sets
        #self.disjoint_sets(self.affordance_multiplex,self.method)

        #Arrangements
        arrangements=0
        while arrangements<self.num_arrangements:
            print 'arrangement',arrangements
            arrangements+=1
            
            e_route=pd.DataFrame(index=self.ship.edges(),columns=self.affordance_edges)
            e_route=e_route.fillna(0.0)
            #print self.e_route

            n_location=pd.DataFrame(index=self.ship.nodes(),columns=self.affordance_nodes)
            n_location=n_location.fillna(0.0)
            #print self.n_location

            for n in self.affordance_multiplex.global_nodes():
                data=self.affordance_multiplex.global_node[n]
                n_loc=data['loc_possible'][-1]
                #print n_loc
                data['loc'].append(n_loc)
                #print data
                self.affordance_multiplex.edit_plex_node(n,data)
                n_location.ix[n_loc,n]+=1
                
            n_location['total']=n_location.sum(1)
            self.arr_n_list_method.append(n_location)
                                      
            for plex,name in zip(self.affordance_multiplex.plexes(),self.affordance_multiplex.names()):
                for u,v in plex.edges():
                    source=self.affordance_multiplex.global_node[u]['loc'][-1]
                    target=self.affordance_multiplex.global_node[v]['loc'][-1]
                        
                    e_list=[]
                    #print source, target
                    disjoint_paths=self.disjoint_sets[source][target]
                    max_redundancy=len(max(disjoint_paths,key=len))
                    if self.redundancy>max_redundancy:
                        poss_path_sets=filter(lambda x: len(x)==max_redundancy,disjoint_paths)
                    else:
                        poss_path_sets=filter(lambda x: len(x)==self.redundancy,disjoint_paths)
                    path_sets=poss_path_sets[numpy.random.randint(len(poss_path_sets))]
                    for path in path_sets:
                        e_list_t=[]
                        for i in range(len(path)-1):
                            e=(path[i],path[i+1])

                            if not e:
                                continue
                            elif e in e_route.index:
                                self.e_att=(e,(u,v,name))
                                e_route.ix[e,(u,v,name)] += 1
                                e_list_t.append(e)

                            else:
                                self.e_att=(e[::-1],(u,v,name))
                                e_route.ix[e[::-1],(u,v,name)] += 1
                                e_list_t.append(e[::-1])
                        e_list.append(e_list_t)
                    plex[u][v]['paths'].append(e_list)
            
            e_route['total']=e_route.sum(1)
            #get types on edges
            e_route['num_types']=numpy.nan
            for e in e_route.index:
                types=[]
                for a in e_route.columns:
                    if a=='total':
                        continue
                    if (e_route[a][e]>0) and (a[2] not in types):
                        types.append(a[2])
                e_route['num_types'][e]=len(types)
            
            e_route['c_f']=e_route.apply(lambda row: 2**row['total'],axis=1)                
            e_route['c_f_t']=e_route.apply(lambda row: 2**row['num_types'],axis=1)
            self.arr_e_list_method.append(e_route)
            
        return self.arr_n_list_method, self.arr_e_list_method
    
    def k_shortest_paths(self,G, source, target, k=1, weight='weight'):
    
        if source == target:
            return ([0], [[source]]) 

        length, path = nx.single_source_dijkstra(G, source, target, weight=weight)
        if target not in length:
            raise nx.NetworkXNoPath("node %s not reachable from %s" % (source, target))

        lengths = [length[target]]
        paths = [path[target]]
        c = count()        
        B = []                        
        G_original = G.copy()    

        for i in range(1, k):
            for j in range(len(paths[-1]) - 1):            
                spur_node = paths[-1][j]
                root_path = paths[-1][:j + 1]

                edges_removed = []
                for c_path in paths:
                    if len(c_path) > j and root_path == c_path[:j + 1]:
                        u = c_path[j]
                        v = c_path[j + 1]
                        if G.has_edge(u, v):
                            edge_attr = G.edge[u][v]
                            G.remove_edge(u, v)
                            edges_removed.append((u, v, edge_attr))

                for n in range(len(root_path) - 1):
                    node = root_path[n]
                    # out-edges
                    for u, v, edge_attr in G.edges_iter(node, data=True):
                        G.remove_edge(u, v)
                        edges_removed.append((u, v, edge_attr))

                    if G.is_directed():
                        # in-edges
                        for u, v, edge_attr in G.in_edges_iter(node, data=True):
                            G.remove_edge(u, v)
                            edges_removed.append((u, v, edge_attr))

                spur_path_length, spur_path = nx.single_source_dijkstra(G, spur_node, target, weight=weight)            
                if target in spur_path and spur_path[target]:
                    total_path = root_path[:-1] + spur_path[target]
                    total_path_length = self.get_path_length(G_original, root_path, weight) + spur_path_length[target]                
                    heappush(B, (total_path_length, next(c), total_path))

                for e in edges_removed:
                    u, v, edge_attr = e
                    G.add_edge(u, v, edge_attr)

            if B:
                (l, _, p) = heappop(B)        
                lengths.append(l)
                paths.append(p)
            else:
                break

        return (lengths, paths)
 
    def get_path_length(self,G, path, weight='weight'):
        length = 0
        if len(path) > 1:
            for i in range(len(path) - 1):
                u = path[i]
                v = path[i + 1]

                length += G.edge[u][v].get(weight, 1)

        return length
    
    def comb(self,input, lst = [], lset = set()):
        if lst:
            yield lst
        for i, el in enumerate(input):
            if lset.isdisjoint(el):
                for out in self.comb(input[i+1:], lst + [el], lset | set(el)):
                    yield out

    def get_incidence(self, n_locations,e_routes):
        
        self.arr_a_list=[]
        self.arr_ae_list=[]
        
        for arr in range(len(n_locations)): 
        
            a_incidence=pd.DataFrame(index=self.affordance_nodes,columns=self.affordance_nodes)
            a_incidence=a_incidence.fillna(0.0)
            #print self.a_incidence

            ae_incidence=pd.DataFrame(index=self.affordance_edges,columns=self.affordance_edges)
            ae_incidence=ae_incidence.fillna(0)
            #print self.ae_incidence
            #post-process locations
            for n1 in self.affordance_multiplex.global_nodes(): #get affordance incidence
                for n2 in self.affordance_multiplex.global_nodes():
                    #if n1==n2:
                        #continue
                    if n_locations[arr].index[n_locations[arr][n1]==1]==n_locations[arr].index[n_locations[arr][n2]==1]:
                        a_incidence.ix[n1,n2]+=1
                        #a_incidence.ix[n2,n1]+=1
            a_incidence['total']=a_incidence.sum(1)
            self.arr_a_list.append(a_incidence)
            
            #post-process edges
            for e1 in e_routes[arr].columns:
                length=e_routes[arr][e1].sum()
                #print length
                for i in e_routes[arr].index[e_routes[arr][e1]==1]:
                    for e2 in e_routes[arr].columns:
                        #print e_routes[-1][e2][i]
                        if isinstance(e1, str) or isinstance(e2, str): #e1==('total' or 'c_f') or e2==('total' or 'c_f'):
                            continue
                        if e1==e2:
                            continue
                        if e_routes[arr][e2][i]==1:
                            ae_incidence.ix[e1,e2]+=1.0#/length
                       
            ae_incidence['total']=ae_incidence.sum(1)
            ae_incidence['diff_total']=0.0
            #print ae_incidence
            for i in ae_incidence.index:
                diff_total=0
                for c in ae_incidence.columns:
                    if isinstance(c, str): #c==('total' or 'diff_total' or 'c_f'):
                        continue
                    if i[2] != c[2]:
                        #print i,c,ae_incidence[c][i]
                        diff_total+=float(ae_incidence[c][i])
                ae_incidence['diff_total'][i]=diff_total
            self.arr_ae_list.append(ae_incidence)
            
        return self.arr_a_list, self.arr_ae_list
    
    def node_complexity(self,n_location,e_route):
        self.arr_n_cmplx_list=[]
        for arr in range(len(n_location)):
            n_cmplx=pd.DataFrame(index=n_location[arr].index,columns=['c_a','c_e','c_e_t','c_f','c_f_t','p'])
            n_cmplx=n_cmplx.fillna(0.0)
            for n in n_location[arr].index:
                types=[] #types using node
                a_pairs=[] #affordance pairs using node
                #print n
                #print n_location[arr]['total'][n]
                c_a=2.0**n_location[arr]['total'][n]
                #print n_cmplx['c_a'][n]
                #c_e=-1.0+self.redundancy*n_location[arr]['total'][n] #ORIGINAL
                c_e=self.redundancy*n_location[arr]['total'][n]-1.0 #subtract 1 because it will be counted once below
                for a in e_route[arr].columns: #count edges that use node
                    if type(a) is tuple:
                        path=e_route[arr].index[e_route[arr][a]==1]
                        for e in path:
                            if n in e:
                                if a[2] not in types:
                                    types.append(a[2])
                                if (a[0],a[1]) not in a_pairs:
                                    c_e+=1
                                    a_pairs.append((a[0],a[1]))
                                break #count each pair only once, prevents double count from head+tail of edges mid-route
                #print c_e
                if len(types)==8.0:
                    print types
                n_cmplx['c_a'][n]=c_a
                n_cmplx['c_e'][n]=c_e
                n_cmplx['c_e_t'][n]=len(types)
                n_cmplx['c_f'][n]=c_a*2.0**c_e
                n_cmplx['c_f_t'][n]=c_a*2.0**len(types)
                n_cmplx['p'][n]=2.0**(-(c_e-c_a))
               
            self.arr_n_cmplx_list.append(n_cmplx)
            
        return self.arr_n_cmplx_list
    
    def affordance_complexity(self,a_incidence,ae_incidence):
        self.arr_a_cmplx_list=[]
        for arr in range(len(a_incidence)):
            #print a_incidence[arr]
            #print a_incidence[arr].index
            a_cmplx=pd.DataFrame(index=a_incidence[arr].index,columns=['c_a','c_e','c_d','p'])
            a_cmplx=a_cmplx.fillna(0.0)
            for a in a_incidence[arr].index:
                c_a=2.0**float(a_incidence[arr]['total'][a])
                c_e=0.0
                c_e_d=0.0
                for ae in ae_incidence[arr].index:
                    if a in ae: #is affrodance in the connection
                        #print ae_incidence[arr]['total'][ae]
                        c_e+=float(ae_incidence[arr]['total'][ae])
                        #print c_e
                        c_e_d+=float(ae_incidence[arr]['diff_total'][ae])
                #c_f=c_a*2.0**c_e
                p=2.0**(-(c_e_d-c_a))
                #print a
                #print float(c_a)
                #print float(c_e)
                #print float(c_f)
                #print p
                #print a_cmplx
                #print 'c_e test',c_e
                a_cmplx['c_a'][a]=float(c_a)
                a_cmplx['c_e'][a]=float(c_e)
                #print a_cmplx['c_e'][a]
                a_cmplx['c_d'][a]=float(c_e_d)
                a_cmplx['p'][a]=p
                
            self.arr_a_cmplx_list.append(a_cmplx)
        return self.arr_a_cmplx_list
    
    def total_interactions(self,node_complx_table,e_routes): #input is output from node_complexity function
        self.total_int_list=[]
        for arr in range(len(node_complx_table)):
            types=[node_complx_table[arr]['c_f_t'].sum(0)+e_routes[arr]['c_f_t'].sum(0),
                    node_complx_table[arr]['c_f_t'].sum(0),
                    e_routes[arr]['c_f_t'].sum(0)]
            
            totals=[node_complx_table[arr]['c_f'].sum(0)+e_routes[arr]['c_f'].sum(0),
                    node_complx_table[arr]['c_f'].sum(0),
                    e_routes[arr]['c_f'].sum(0)]
            
            self.total_int_list.append((types,totals))
                #node_complx_table[arr]['c_f_t'].sum(0)+e_routes[arr]['c_f_t'].sum(0)                        
                #+e_routes[arr]['c_f'].sum(0))
                                        #node_complx_table[arr]['c_f'].sum(0)))
            #((node_complx_table[arr]['c_f'].sum(0),node_complx_table[arr]['c_f_t'].sum(0)),
                                         #(e_routes[arr]['c_f'].sum(0),e_routes[arr]['c_f_t'].sum(0)))
            
        return self.total_int_list
                
        
            
            
        

In [4]:
test_ship=MultiPlex()
test_ship.add_plex('E')
test_ship.add_plex('M')
test_ship.add_plex('I')
test_ship.add_plex('H')
test_ship.add_plex_node('store_E',data={'loc':[],'loc_possible':[(3,3,0)]})
test_ship.add_plex_node('store_M',data={'loc':[],'loc_possible':[(6,3,3)]})
test_ship.add_plex_node('conv_m',data={'loc':[],'loc_possible':[(4,3,1)]})
test_ship.add_plex_node('propulsion',data={'loc':[],'loc_possible':[(1,3,1)]})
test_ship.add_plex_node('conv_e',data={'loc':[],'loc_possible':[(5,3,1)]})
test_ship.add_plex_node('control',data={'loc':[],'loc_possible':[(5,3,9)]})
test_ship.add_plex_node('move_int',data={'loc':[],'loc_possible':[(7,3,3)]})
test_ship.add_plex_node('move_ext',data={'loc':[],'loc_possible':[(5,3,7)]})
test_ship.add_plex_edge(('store_E','conv_m','E'),data={'paths':[]})
test_ship.add_plex_edge(('conv_m','propulsion','E'),data={'paths':[]})
test_ship.add_plex_edge(('store_E','conv_e','E'),data={'paths':[]})
test_ship.add_plex_edge(('conv_e','control','E'),data={'paths':[]})
test_ship.add_plex_edge(('conv_e','move_int','E'),data={'paths':[]})
test_ship.add_plex_edge(('conv_e','move_ext','E'),data={'paths':[]})
test_ship.add_plex_edge(('store_M','move_int','M'),data={'paths':[]})
#test_ship.add_plex_edge(('move_int','move_ext','M'),data={'paths':[]})
test_ship.add_plex_edge(('control','move_ext','I'),data={'paths':[]})
test_ship.add_plex_edge(('control','move_int','I'),data={'paths':[]})
test_ship.add_plex_edge(('control','propulsion','I'),data={'paths':[]})
test_ship.add_plex_edge(('control','conv_e','I'),data={'paths':[]})
test_ship.add_plex_edge(('control','conv_m','I'),data={'paths':[]})
test_ship.add_plex_edge(('control','conv_m','H'),data={'paths':[]})
test_ship.add_plex_edge(('control','move_ext','H'),data={'paths':[]})
test_ship.add_plex_edge(('control','move_int','H'),data={'paths':[]})
test_ship.add_plex_edge(('control','propulsion','H'),data={'paths':[]})
test_ship.add_plex_edge(('control','conv_e','H'),data={'paths':[]})
test_ship.add_plex_edge(('conv_e','conv_m','H'),data={'paths':[]})
test_ship.add_plex_edge(('conv_e','store_M','H'),data={'paths':[]})
test_ship.add_plex_edge(('conv_e','move_ext','H'),data={'paths':[]})
test_ship.add_plex_edge(('conv_e','move_int','H'),data={'paths':[]})
test_ship.add_plex_edge(('conv_m','store_M','H'),data={'paths':[]})
test_ship.add_plex_edge(('conv_m','move_ext','H'),data={'paths':[]})
test_ship.add_plex_edge(('conv_m','move_int','H'),data={'paths':[]})
test_ship.add_plex_edge(('store_M','move_ext','H'),data={'paths':[]})
test_ship.add_plex_edge(('store_M','move_int','H'),data={'paths':[]})
test_ship.add_plex_edge(('move_ext','move_int','H'),data={'paths':[]})

for plex,name in zip(test_ship.plexes(),test_ship.names()):
    print name
    print plex.nodes(data=True)
    print plex.edges(data=True)


E
[('store_M', {'loc': [], 'loc_possible': [(6, 3, 3)]}), ('control', {'loc': [], 'loc_possible': [(5, 3, 9)]}), ('move_int', {'loc': [], 'loc_possible': [(7, 3, 3)]}), ('conv_m', {'loc': [], 'loc_possible': [(4, 3, 1)]}), ('store_E', {'loc': [], 'loc_possible': [(3, 3, 0)]}), ('propulsion', {'loc': [], 'loc_possible': [(1, 3, 1)]}), ('conv_e', {'loc': [], 'loc_possible': [(5, 3, 1)]}), ('move_ext', {'loc': [], 'loc_possible': [(5, 3, 7)]})]
[('control', 'conv_e', {'paths': []}), ('move_int', 'conv_e', {'paths': []}), ('conv_m', 'store_E', {'paths': []}), ('conv_m', 'propulsion', {'paths': []}), ('store_E', 'conv_e', {'paths': []}), ('conv_e', 'move_ext', {'paths': []})]
M
[('store_M', {'loc': [], 'loc_possible': [(6, 3, 3)]}), ('control', {'loc': [], 'loc_possible': [(5, 3, 9)]}), ('move_int', {'loc': [], 'loc_possible': [(7, 3, 3)]}), ('conv_m', {'loc': [], 'loc_possible': [(4, 3, 1)]}), ('store_E', {'loc': [], 'loc_possible': [(3, 3, 0)]}), ('propulsion', {'loc': [], 'loc_possible':

In [10]:
#Setup 10KT Warship
warship=MultiPlex()
warship.add_plex('P')
warship.add_plex('EM_P')
warship.add_plex('C')
warship.add_plex('I')
warship.add_plex('H')

#Define nodes
warship.add_plex_node('GS_1',data={'loc':[],'loc_possible':[(2,2,0)]})
warship.add_plex_node('GS_2',data={'loc':[],'loc_possible':[(3,0,0)]})
warship.add_plex_node('AUX_1',data={'loc':[],'loc_possible':[(4,2,0)]})
warship.add_plex_node('AUX_2',data={'loc':[],'loc_possible':[(5,0,0)]})
warship.add_plex_node('EM_1',data={'loc':[],'loc_possible':[(1,0,0)]})
warship.add_plex_node('EM_2',data={'loc':[],'loc_possible':[(2,2,0)]})
warship.add_plex_node('Chiller_1',data={'loc':[],'loc_possible':[(4,1,0)]})
warship.add_plex_node('Chiller_2',data={'loc':[],'loc_possible':[(5,1,0)]})
warship.add_plex_node('Sonar',data={'loc':[],'loc_possible':[(6,1,0)]})
warship.add_plex_node('CIC_1',data={'loc':[],'loc_possible':[(4,1,4)]})
warship.add_plex_node('CIC_2',data={'loc':[],'loc_possible':[(2,1,3)]})
warship.add_plex_node('ADS_1',data={'loc':[],'loc_possible':[(0,0,3)]})
warship.add_plex_node('ADS_2',data={'loc':[],'loc_possible':[(0,2,3)]})
warship.add_plex_node('ADS_3',data={'loc':[],'loc_possible':[(7,0,3)]})
warship.add_plex_node('ADS_4',data={'loc':[],'loc_possible':[(7,2,3)]})
warship.add_plex_node('VLS',data={'loc':[],'loc_possible':[(1,1,3)]})
warship.add_plex_node('Radar',data={'loc':[],'loc_possible':[(4,1,5)]})
warship.add_plex_node('LASER_1',data={'loc':[],'loc_possible':[(2,1,5)]})
warship.add_plex_node('LASER_2',data={'loc':[],'loc_possible':[(5,1,5)]})
warship.add_plex_node('Railgun',data={'loc':[],'loc_possible':[(6,1,3)]})
warship.add_plex_node('Hotel_1',data={'loc':[],'loc_possible':[(2,0,2)]})
warship.add_plex_node('Hotel_2',data={'loc':[],'loc_possible':[(3,0,2)]})
warship.add_plex_node('Hotel_3',data={'loc':[],'loc_possible':[(4,0,2)]})
warship.add_plex_node('Hotel_4',data={'loc':[],'loc_possible':[(5,0,2)]})
warship.add_plex_node('Machinery_1',data={'loc':[],'loc_possible':[(7,0,1)]})
warship.add_plex_node('Machinery_2',data={'loc':[],'loc_possible':[(7,0,2)]})
warship.add_plex_node('Machinery_3',data={'loc':[],'loc_possible':[(7,0,1)]})

#Define edges
#EM_P
warship.add_plex_edge(('GS_1','EM_1','EM_P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','EM_2','EM_P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','EM_1','EM_P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','EM_2','EM_P'),data={'paths':[]})

#P
warship.add_plex_edge(('AUX_1','Chiller_1','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_2','Chiller_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','CIC_1','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','CIC_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','CIC_1','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','CIC_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','Sonar','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','Sonar','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_1','Sonar','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_2','Sonar','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','ADS_1','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','ADS_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','ADS_3','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','ADS_4','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','ADS_1','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','ADS_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','ADS_3','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','ADS_4','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','VLS','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','VLS','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','LASER_1','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','LASER_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','LASER_1','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','LASER_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','Radar','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','Radar','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_1','Radar','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_2','Radar','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','Railgun','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','Railgun','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','Hotel_1','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','Hotel_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','Hotel_3','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','Hotel_4','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','Hotel_1','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','Hotel_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','Hotel_3','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','Hotel_4','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_1','Hotel_1','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_1','Hotel_2','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_1','Hotel_3','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_1','Hotel_4','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_2','Hotel_1','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_2','Hotel_2','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_2','Hotel_3','P'),data={'paths':[]})
warship.add_plex_edge(('AUX_2','Hotel_4','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','Machinery_1','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','Machinery_1','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','Machinery_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','Machinery_2','P'),data={'paths':[]})
warship.add_plex_edge(('GS_1','Machinery_3','P'),data={'paths':[]})
warship.add_plex_edge(('GS_2','Machinery_3','P'),data={'paths':[]})

#C
warship.add_plex_edge(('Chiller_1','Radar','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','Radar','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','Sonar','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','Sonar','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','Railgun','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','Railgun','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','VLS','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','VLS','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','LASER_1','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','LASER_2','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','LASER_1','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','LASER_2','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','CIC_1','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','CIC_2','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','CIC_1','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','CIC_2','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','Hotel_1','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','Hotel_2','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','Hotel_3','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_1','Hotel_4','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','Hotel_1','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','Hotel_2','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','Hotel_3','C'),data={'paths':[]})
warship.add_plex_edge(('Chiller_2','Hotel_4','C'),data={'paths':[]})

#I
warship.add_plex_edge(('Radar','CIC_1','I'),data={'paths':[]})
warship.add_plex_edge(('Radar','CIC_2','I'),data={'paths':[]})
warship.add_plex_edge(('Sonar','CIC_1','I'),data={'paths':[]})
warship.add_plex_edge(('Sonar','CIC_2','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_1','VLS','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_2','VLS','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_1','LASER_1','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_1','LASER_2','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_2','LASER_1','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_2','LASER_2','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_1','Railgun','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_2','Railgun','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_1','ADS_1','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_1','ADS_2','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_1','ADS_3','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_1','ADS_4','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_2','ADS_1','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_2','ADS_2','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_2','ADS_3','I'),data={'paths':[]})
warship.add_plex_edge(('CIC_2','ADS_4','I'),data={'paths':[]})

#H
#for n1 in warship.global_node:
 #   for n2 in warship.global_node:
  #      if n1==n2:
   #         continue
    #    if (n2,n1) in warship.plex[3].edges():
     #       continue
      #  warship.add_plex_edge((n1,n2,'H'),data={'paths':[]})

In [6]:
ship=ShipNET([8,3,6])
for x in range(2):
    for z in range (4,6):
        for y in range(3):
            ship.remove_ship_node((x,y,z))
for x in range(6,8):
    for z in range (4,6):
        for y in range(3):
            ship.remove_ship_node((x,y,z))
ship.remove_ship_node((0,0,0))
ship.remove_ship_node((0,1,0))
ship.remove_ship_node((0,2,0))

ship.set_dc_config(['T0','T1','T3','T4','T5','T6','T7','T8',],3,exception=['EM_P','H']) #'EM_P','H'
ship.gen_disjoint_sets(warship,'shortest',k=100)

print ship.ship_dual.nodes()

['H1', 'H6', 'T8', 'T6', 'T7', 'T4', 'T5', 'T2', 'T3', 'H4', 'T1', 'H2', 'L2', 'T0', 'L0', 'L1', 'H3', 'H5', 'H0', 'L3']


In [11]:
ship=ShipNET([8,3,6])
for x in range(2):
    for z in range (4,6):
        for y in range(3):
            ship.remove_ship_node((x,y,z))
for x in range(6,8):
    for z in range (4,6):
        for y in range(3):
            ship.remove_ship_node((x,y,z))
ship.remove_ship_node((0,0,0))
ship.remove_ship_node((0,1,0))
ship.remove_ship_node((0,2,0))

cs_type=list(itertools.chain(["num_bh","height_dc","redundancy",
                         "interactions_type","node_type","edge_type"],
                        ship.ship.nodes(),ship.ship.edges()))
cs_all=list(itertools.chain(["num_bh","height_dc","redundancy",
                         "interactions_total","node_total","edge_total"],
                        ship.ship.nodes(),ship.ship.edges()))
output_path="warship_no_h_split"
arr_data_type=[]
arr_data_all=[]
runs=1
#bh_setups=[[],
           #['T4'],
           #['T2','T6'],
           #['T2','T4','T6'],
           #['T1','T3','T5','T7'],
           #['T1','T2','T4','T6','T7'],
           #['T1','T2','T3','T5','T6','T7'],
           #['T1','T2','T3','T4','T5','T6','T7']]
            
bh_setups=[['T1','T2','T3','T4','T5','T6','T7']]

for bh in bh_setups:
        for bh_height in range(3,4):
            #set up new ship
            ship=ShipNET([8,3,6])
            for x in range(2):
                for z in range (4,6):
                    for y in range(3):
                        ship.remove_ship_node((x,y,z))
            for x in range(6,8):
                for z in range (4,6):
                    for y in range(3):
                        ship.remove_ship_node((x,y,z))
            ship.remove_ship_node((0,0,0))
            ship.remove_ship_node((0,1,0))
            ship.remove_ship_node((0,2,0))

            ship.set_dc_config(bh,bh_height,exception=['EM_P','H']) #'EM_P','H'
            ship.gen_disjoint_sets(warship,'shortest',k=100)
            
            for red in range(2,3):
                print len(bh),bh_height,red
                c=0
                while c<runs:
                    c+=1
                    loc_t,route_t=ship.affordance_routing(warship,1,method='shortest',redundancy=red)
                    #method[key]['loc']=loc_t
                    #method[key]['route']=route_t
                    a_t,ae_t=ship.get_incidence(loc_t,route_t)
                    n_c_t=ship.node_complexity(loc_t,route_t)
                    a_c_t=ship.affordance_complexity(a_t,ae_t)
                    t_int=ship.total_interactions(n_c_t,route_t)
                    #method[key]['a_inc']=a_t
                    #method[key]['ae_inc']=ae_t
                    #method[key]['n_cmplx']=n_c_t
                    #method[key]['a_cmplx']=a_c_t
                    #method[key]['t_int']=t_int
                    arr_data_type.append(list(itertools.chain([ship.num_bh],[ship.height_dc],[ship.redundancy],
                                                         t_int[0][0],n_c_t[0]['c_f_t'],route_t[0]['c_f_t'])))
                    arr_data_all.append(list(itertools.chain([ship.num_bh],[ship.height_dc],[ship.redundancy],
                                                         t_int[0][1],n_c_t[0]['c_f'],route_t[0]['c_f'])))
                    #store_model(ship,output_path)
                    
t_type=pd.DataFrame(arr_data_type,columns=cs_type)
t_all=pd.DataFrame(arr_data_all,columns=cs_all)
#t.to_csv(os.path.join(output_path, "arrangement_series.csv"))

7 3 2
arrangement 0


In [13]:
print t_all


   num_bh  height_dc  redundancy  interactions_total    node_total  \
0       7          3           2        4.446097e+12  4.445970e+12   

   edge_total  (4, 2, 2)   (3, 0, 3)  (7, 1, 3)  (2, 2, 4)    ...      \
0   127162037         16  4294967296        128          2    ...       

   ((0, 1, 1), (1, 1, 1))  ((0, 1, 3), (1, 1, 3))  ((5, 1, 4), (5, 2, 4))  \
0                       1                      32                       2   

   ((4, 0, 1), (3, 0, 1))  ((5, 2, 4), (4, 2, 4))  ((5, 2, 4), (5, 2, 5))  \
0                       1                       1                       1   

   ((6, 2, 3), (6, 1, 3))  ((6, 2, 1), (6, 1, 1))  ((2, 1, 2), (2, 2, 2))  \
0                       4                       1                       2   

   ((2, 1, 0), (2, 2, 0))  
0                  524288  

[1 rows x 393 columns]


In [183]:
ship=ShipNET([10,5,10])
for x in range(3):
    for z in range (7,10):
        for y in range(5):
            ship.remove_ship_node((x,y,z))
for x in range(6,10):
    for z in range (7,10):
        for y in range(5):
            ship.remove_ship_node((x,y,z))        
ship.set_dc_config(['T1','T2','T3','T4','T5','T6','T7','T8','T9'],0,exception=['H'])
#print ship.dual_streets
loc=[]
route=[]
method={'k-shortest':{'loc':[],'route':[]}} #,'k-shortest':{'loc':[],'route':[]}}
#method={'random':{'loc':[],'route':[]},'shortest':{'loc':[],'route':[]},'simple':{'loc':[],'route':[]}}
ship.gen_disjoint_sets(test_ship,'shortest')

#1030-### for k=10000

KeyError: 'E'

In [19]:
cs=list(itertools.chain(["num_bh","height_dc","redundancy",
                         "interactions_type","interactions_total",
                         "node_type","node_total",
                         "edge_type","edge_total"],ship.ship.nodes(),ship.ship.edges()))
#print cs

#for key in method:
arr_data=[]
c=0

while c<10:
    c+=1
    loc_t,route_t=ship.affordance_routing(warship,1,method='k-shortest',redundancy=2)
    #method[key]['loc']=loc_t
    #method[key]['route']=route_t
    a_t,ae_t=ship.get_incidence(loc_t,route_t)
    n_c_t=ship.node_complexity(loc_t,route_t)
    a_c_t=ship.affordance_complexity(a_t,ae_t)
    t_int=ship.total_interactions(n_c_t,route_t)
    #method[key]['a_inc']=a_t
    #method[key]['ae_inc']=ae_t
    #method[key]['n_cmplx']=n_c_t
    #method[key]['a_cmplx']=a_c_t
    #method[key]['t_int']=t_int
    #print t_int
    arr_data.append(list(itertools.chain([ship.num_bh],[ship.height_dc],[ship.redundancy],
                                         t_int[0],
                                         zip(n_c_t[0]['c_f_t'],n_c_t[0]['c_f']),
                                         zip(route_t[0]['c_f_t'],route_t[0]['c_f']))))

t_3=pd.DataFrame(arr_data,columns=cs)    
#print method

#loc_s,route_s=ship.affordance_routing_simple(test_ship,1,1.1)

#a,ae=ship.get_incidence(loc,route)
#n_c=ship.node_complexity(loc,route)
#a_c=ship.affordance_complexity(a,ae)
#print a_c
#print ship.arr_e_list
#print ship.arr_n_list
#print ship.arr_a_list
#print ship.arr_ae_list

arrangement 0
[3985.0, 3.456081163279557e+37, 1752.0, 3.4560811632794966e+37, 2233.0, 6.0446351237339579e+23]
arrangement 0
[3839.0, 5.5755478840500672e+42, 1682.0, 5.5755478840500672e+42, 2157.0, 1.511699095957995e+23]
arrangement 0
[4066.0, 1.7423536503283991e+41, 1774.0, 1.7423536503283991e+41, 2292.0, 1.5163793665937167e+23]
arrangement 0
[4006.0, 1.7944751500215802e+37, 1744.0, 1.7944751500215802e+37, 2262.0, 4.490123269297849e+20]
arrangement 0
[4108.0, 5.4681122913498226e+39, 1818.0, 5.4681122913498226e+39, 2290.0, 2.4207312008527355e+22]
arrangement 0
[3801.0, 1.0135429933450333e+38, 1682.0, 1.0135429933450333e+38, 2119.0, 2.3653014339127308e+21]
arrangement 0
[3843.0, 5.4502557074480413e+39, 1680.0, 5.4502557074480413e+39, 2163.0, 4.7985782254043063e+21]
arrangement 0
[4242.0, 1.781230497906404e+38, 1910.0, 1.781230497906404e+38, 2332.0, 1.2010425220515575e+21]
arrangement 0
[4034.0, 2.6053666168439472e+38, 1818.0, 2.6053666168439472e+38, 2216.0, 1.8779639336927019e+20]
arrang

In [22]:
#print method['k-shortest']['t_int']
print t[["interactions_type","interactions_total","node_type","node_total","edge_type","edge_total"]]
print t_3[["interactions_type","interactions_total","node_type","node_total","edge_type","edge_total"]]

   interactions_type  interactions_total  node_type    node_total  edge_type  \
0               3976        9.044024e+37       1810  9.044024e+37       2166   
1               3987        5.790130e+39       1790  5.790130e+39       2197   
2               3909        1.488948e+38       1734  1.488948e+38       2175   
3               3955        8.719878e+40       1742  8.719878e+40       2213   
4               3976        1.373269e+39       1734  1.373269e+39       2242   
5               3892        7.665086e+38       1690  7.665086e+38       2202   
6               3874        6.940649e+38       1710  6.940649e+38       2164   
7               4090        8.546148e+37       1848  8.546148e+37       2242   
8               3874        4.084388e+39       1704  4.084388e+39       2170   
9               4052        1.026499e+39       1784  1.026499e+39       2268   

     edge_total  
0  7.103430e+21  
1  3.028250e+23  
2  2.021930e+22  
3  1.561442e+23  
4  4.946201e+21  
5  6.092776

In [242]:
#print a_t
#print ae_t
print n_c_t[0]
print n_c_t[0]['c_e_t'].argmax()
#print a_c_t
#print loc_t
print zip(n_c_t[0]['c_f_t'],n_c_t[0]['c_f'])

           c_a  c_e  c_e_t           c_f  c_f_t             p
(4, 2, 2)    1   15      2  3.276800e+04      4  6.103516e-05
(3, 0, 3)    1   29      4  5.368709e+08     16  3.725290e-09
(7, 1, 3)    1   24      3  1.677722e+07      8  1.192093e-07
(2, 2, 4)    1    5      2  3.200000e+01      4  6.250000e-02
(7, 2, 3)    2   23      3  1.677722e+07     16  4.768372e-07
(6, 0, 0)    1   18      3  2.621440e+05      8  7.629395e-06
(5, 2, 1)    1    0      1  1.000000e+00      2  2.000000e+00
(7, 1, 1)    1    2      1  4.000000e+00      2  5.000000e-01
(0, 2, 1)    1    0      1  1.000000e+00      2  2.000000e+00
(7, 2, 1)    1    4      2  1.600000e+01      4  1.250000e-01
(6, 0, 2)    1    4      2  1.600000e+01      4  1.250000e-01
(5, 2, 0)    1   12      3  4.096000e+03      8  4.882812e-04
(5, 2, 3)    1   17      3  1.310720e+05      8  1.525879e-05
(0, 2, 3)    2   28      3  5.368709e+08     16  1.490116e-08
(3, 2, 4)    1    5      4  3.200000e+01     16  6.250000e-02
(3, 1, 4

In [241]:
print route_t[0]
print n_c_t[0]['c_f'].sum(0)
print route_t[0]['num_types'].argmax()

                        (LASER_2, DG_1, P)  (LASER_1, DG_2, P)  \
((4, 2, 2), (4, 1, 2))                   0                   0   
((4, 2, 2), (4, 2, 3))                   0                   0   
((4, 2, 2), (3, 2, 2))                   1                   0   
((4, 2, 2), (5, 2, 2))                   1                   0   
((4, 2, 2), (4, 2, 1))                   0                   0   
((3, 0, 3), (3, 0, 2))                   0                   0   
((3, 0, 3), (4, 0, 3))                   0                   0   
((3, 0, 3), (2, 0, 3))                   0                   0   
((3, 0, 3), (3, 0, 4))                   0                   0   
((3, 0, 3), (3, 1, 3))                   0                   0   
((7, 1, 3), (6, 1, 3))                   0                   0   
((7, 1, 3), (7, 2, 3))                   0                   0   
((7, 1, 3), (7, 0, 3))                   0                   0   
((7, 1, 3), (7, 1, 2))                   0                   0   
((2, 2, 4)

In [13]:
print ship.ship_dual.nodes()

['T10', 'T6', 'T8', 'T9', 'H10', 'T7', 'T4', 'T5', 'T2', 'T3', 'T0', 'T1', 'L4', 'L5', 'L2', 'L3', 'L0', 'L1', 'H8', 'H9', 'H2', 'H3', 'H0', 'H1', 'H6', 'H7', 'H4', 'H5']


In [7]:
t=ShipNET([2,2,2])
t.set_dc_config(['T1'],'H1',exception=['H'])
print t.ship_dual.nodes()
t.transfer_graphs()
for g in t.transfer_types:
    print g
    print t.transfer_types[g].edges()

((1, 1, 0), (1, 0, 0)) False
((1, 1, 0), (0, 1, 0)) True
((1, 1, 0), (1, 1, 1)) False
((0, 1, 1), (0, 1, 0)) False
((0, 1, 1), (0, 0, 1)) False
((0, 1, 1), (1, 1, 1)) False
((1, 0, 0), (0, 0, 0)) True
((1, 0, 0), (1, 0, 1)) False
((0, 0, 1), (0, 0, 0)) False
((0, 0, 1), (1, 0, 1)) False
((1, 0, 1), (1, 1, 1)) False
((0, 0, 0), (0, 1, 0)) False
['H2', 'H0', 'H1', 'T2', 'T0', 'T1', 'L2', 'L0', 'L1']
I
[((1, 1, 0), (1, 0, 0)), ((1, 1, 0), (1, 1, 1)), ((0, 1, 1), (0, 1, 0)), ((0, 1, 1), (0, 0, 1)), ((0, 1, 1), (1, 1, 1)), ((1, 0, 0), (1, 0, 1)), ((0, 0, 1), (0, 0, 0)), ((0, 0, 1), (1, 0, 1)), ((1, 0, 1), (1, 1, 1)), ((0, 0, 0), (0, 1, 0))]
M
[((1, 1, 0), (1, 0, 0)), ((1, 1, 0), (1, 1, 1)), ((0, 1, 1), (0, 1, 0)), ((0, 1, 1), (0, 0, 1)), ((0, 1, 1), (1, 1, 1)), ((1, 0, 0), (1, 0, 1)), ((0, 0, 1), (0, 0, 0)), ((0, 0, 1), (1, 0, 1)), ((1, 0, 1), (1, 1, 1)), ((0, 0, 0), (0, 1, 0))]
E
[((1, 1, 0), (1, 0, 0)), ((1, 1, 0), (1, 1, 1)), ((0, 1, 1), (0, 1, 0)), ((0, 1, 1), (0, 0, 1)), ((0, 1, 1), (1

In [8]:
t=ShipNET([2,2,2])
#t.set_dc_config(['T1'],'H1',exception=['H'])
print t.ship_dual.nodes()
#t.transfer_graphs()


['H2', 'H0', 'H1', 'T2', 'T0', 'T1', 'L2', 'L0', 'L1']


In [197]:
d=pd.DataFrame({'one' : [0.,1., 0, 1.],'two':[1.,0,0,0]})
print d
#d['c_e']=d.apply(lambda row: 2**row['two'],axis=1)
#print d

for e in d.columns:
    #print e
    p= d.index[d[e]==1]
    print p
    for n in p:
        #if d[a][e]==1:
            #print e,a
        print n

   one  two
0    0    1
1    1    0
2    0    0
3    1    0
Int64Index([1, 3], dtype='int64')
1
3
Int64Index([0], dtype='int64')
0


In [31]:
e=1
print e==( 1or 3)

True


In [56]:
for z in range (4,6):
    print z

4
5


In [ ]:
for e1 in e_routes[arr].columns:
                length=e_routes[arr][e1].sum()
                #print length
                for i in e_routes[arr].index[e_routes[arr][e1]==1]: